In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext autotime

time: 373 µs (started: 2021-03-09 09:54:07 +01:00)


In [3]:
%cd ..

/Users/rubenbroekx/Documents/Projects/NumberBatchWrapper
time: 2.19 ms (started: 2021-03-09 09:54:07 +01:00)


# Usage

**TODO: Documentation**

In [4]:
import re
import json

from pathlib import Path
from number_batch_wrapper import Wrapper

time: 116 ms (started: 2021-03-09 09:54:07 +01:00)


In [5]:
wrapper = Wrapper(
    language='en',
    path=Path.home() / 'numberbatch',
    level=5,
)

time: 692 µs (started: 2021-03-09 09:54:07 +01:00)


In [6]:
if not wrapper.is_initialised():
    wrapper.initialise(
        inp_path=Path.home() / 'Downloads'
    )

Extracting 'en'..: 9161913it [01:11, 128348.39it/s]                             
Segmenting 'en'..: 0it [01:04, ?it/s]


time: 2min 17s (started: 2021-03-09 09:54:07 +01:00)


In [7]:
with open(Path.cwd() / 'demos/data.json', 'r') as f:
    data = json.load(f)

time: 1.91 ms (started: 2021-03-09 09:56:24 +01:00)


In [11]:
results = wrapper(data)
results.shape

Embedding: 100%|██████████| 1000/1000 [00:05<00:00, 174.88it/s]


(1000, 300)

time: 6.01 s (started: 2021-03-09 10:00:07 +01:00)
